This notebook reads the tabular data from MOJAVE-17 paper (Lister et al. 2019).

The original table header information is commented.

In [1]:
from astropy.table import Table, join, Column
from astropy.time import Time
from astropy import units as u
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np

# # Local modules
from calc_pa import wrap_angle

In [2]:
align_sou = Table.read("../data/aligned-sou.fits")

In [3]:
souname = align_sou["iers_name"]
# pa_k = align_sou["pa_k_ccl2"]
# pa_ka = align_sou["pa_ka_ccl2"]
r_g = align_sou["ang_sep_g_ccl2"]
pa_g = align_sou["pa_g_ccl2"]

Table 3. 

Columns are as follows: 
- (1) B1950 name
- (2) feature identification number (zero indicates core feature)
- (3) observation epoch
- (4) flux density at 15 GHz in milliJansky
- (5) position offset from the core feature (or map center for the core feature entries) in milliarcseconds
- (6) position angle with respect to the core feature (or map center for the core feature entries) in degrees
- (7) FWHM major axis of fitted Gaussian in milliarcseconds
- (8) axial ratio of fitted Gaussian
- (9) major axis position angle of fitted Gaussian in degrees
- (10) robust feature flag a Individual feature epoch not used in kinematic fits.

In [4]:
table3 = Table.read("../data/mojave17/apjab08eet3_mrt.txt",
                    format="ascii.fixed_width_no_header",
                    col_starts=[0, 9, 12, 14, 26, 34, 40, 46, 52, 57, 61],
                    col_ends=[8, 11, 13, 25, 33, 39, 45, 51, 56, 60, 62],
                    names=["iers_name", "F", "f_F", "ObsEpoch",
                           "I", "r", "PA", "MajAxis", "Ratio", "MajPA", "Robust"])

table3["I"].unit = u.mJy
table3["r"].unit = u.mas
table3["PA"].unit = u.deg
table3["MajAxis"].unit = u.mas
table3["MajPA"].unit = u.deg

table3g = table3.group_by("iers_name")

In [5]:
N = len(table3)
obs_epoch = []

for i in range(N):
    epoch_stri = datetime.strptime(table3["ObsEpoch"][i], "%Y %b %d") 
    obs_epoch.append(epoch_stri.isoformat()) 

x  = Time(obs_epoch)
obs_epoch = Column(x.jyear)
    
table3["ObsEpoch"] = obs_epoch

In [6]:
table3_sub = table3[(table3["ObsEpoch"]<2016)&(table3["ObsEpoch"]>2014)]
table3g = table3_sub.group_by("iers_name")

print("We found the jet feature information of {} AGNs between 2014 and 2016.".format(len(table3g.groups)))

We found the jet feature information of 134 AGNs between 2014 and 2016.


In [7]:
join(align_sou, table3g.groups.keys)

iers_name,ra_err_k,dec_err_k,pos_err_k,ra,dec,ra_err_x,dec_err_x,pos_err_x,dra_k,ddec_k,dra_err_k,ddec_err_k,dra_ddec_cov_k,ang_sep_k,pa_k,nor_ra_k,nor_dec_k,nor_sep_k,dra_k_all1,ddec_k_all1,ang_sep_k_all1,pa_k_all1,nor_ra_k_all1,nor_dec_k_all1,nor_sep_k_all1,dra_k_all2,ddec_k_all2,ang_sep_k_all2,pa_k_all2,nor_ra_k_all2,nor_dec_k_all2,nor_sep_k_all2,dra_k_cln1,ddec_k_cln1,ang_sep_k_cln1,pa_k_cln1,nor_ra_k_cln1,nor_dec_k_cln1,nor_sep_k_cln1,dra_k_cln2,ddec_k_cln2,ang_sep_k_cln2,pa_k_cln2,nor_ra_k_cln2,nor_dec_k_cln2,nor_sep_k_cln2,dra_k_com1,ddec_k_com1,ang_sep_k_com1,pa_k_com1,nor_ra_k_com1,nor_dec_k_com1,nor_sep_k_com1,dra_k_com2,ddec_k_com2,ang_sep_k_com2,pa_k_com2,nor_ra_k_com2,nor_dec_k_com2,nor_sep_k_com2,dra_k_ccl1,ddec_k_ccl1,ang_sep_k_ccl1,pa_k_ccl1,nor_ra_k_ccl1,nor_dec_k_ccl1,nor_sep_k_ccl1,dra_k_ccl2,ddec_k_ccl2,ang_sep_k_ccl2,pa_k_ccl2,nor_ra_k_ccl2,nor_dec_k_ccl2,nor_sep_k_ccl2,ra_err_ka,dec_err_ka,pos_err_ka,dra_ka,ddec_ka,dra_err_ka,ddec_err_ka,dra_ddec_cov_ka,ang_sep_ka,pa_ka,nor_ra_ka,nor_dec_ka,nor_sep_ka,dra_ka_all1,ddec_ka_all1,ang_sep_ka_all1,pa_ka_all1,nor_ra_ka_all1,nor_dec_ka_all1,nor_sep_ka_all1,dra_ka_all2,ddec_ka_all2,ang_sep_ka_all2,pa_ka_all2,nor_ra_ka_all2,nor_dec_ka_all2,nor_sep_ka_all2,dra_ka_cln1,ddec_ka_cln1,ang_sep_ka_cln1,pa_ka_cln1,nor_ra_ka_cln1,nor_dec_ka_cln1,nor_sep_ka_cln1,dra_ka_cln2,ddec_ka_cln2,ang_sep_ka_cln2,pa_ka_cln2,nor_ra_ka_cln2,nor_dec_ka_cln2,nor_sep_ka_cln2,dra_ka_com1,ddec_ka_com1,ang_sep_ka_com1,pa_ka_com1,nor_ra_ka_com1,nor_dec_ka_com1,nor_sep_ka_com1,dra_ka_com2,ddec_ka_com2,ang_sep_ka_com2,pa_ka_com2,nor_ra_ka_com2,nor_dec_ka_com2,nor_sep_ka_com2,dra_ka_ccl1,ddec_ka_ccl1,ang_sep_ka_ccl1,pa_ka_ccl1,nor_ra_ka_ccl1,nor_dec_ka_ccl1,nor_sep_ka_ccl1,dra_ka_ccl2,ddec_ka_ccl2,ang_sep_ka_ccl2,pa_ka_ccl2,nor_ra_ka_ccl2,nor_dec_ka_ccl2,nor_sep_ka_ccl2,ra_err_g,dec_err_g,pos_err_g,dra_g,ddec_g,dra_err_g,ddec_err_g,dra_ddec_cov_g,ang_sep_g,pa_g,nor_ra_g,nor_dec_g,nor_sep_g,dra_g_all1,ddec_g_all1,ang_sep_g_all1,pa_g_all1,nor_ra_g_all1,nor_dec_g_all1,nor_sep_g_all1,dra_g_all2,ddec_g_all2,ang_sep_g_all2,pa_g_all2,nor_ra_g_all2,nor_dec_g_all2,nor_sep_g_all2,dra_g_cln1,ddec_g_cln1,ang_sep_g_cln1,pa_g_cln1,nor_ra_g_cln1,nor_dec_g_cln1,nor_sep_g_cln1,dra_g_cln2,ddec_g_cln2,ang_sep_g_cln2,pa_g_cln2,nor_ra_g_cln2,nor_dec_g_cln2,nor_sep_g_cln2,dra_g_com1,ddec_g_com1,ang_sep_g_com1,pa_g_com1,nor_ra_g_com1,nor_dec_g_com1,nor_sep_g_com1,dra_g_com2,ddec_g_com2,ang_sep_g_com2,pa_g_com2,nor_ra_g_com2,nor_dec_g_com2,nor_sep_g_com2,dra_g_ccl1,ddec_g_ccl1,ang_sep_g_ccl1,pa_g_ccl1,nor_ra_g_ccl1,nor_dec_g_ccl1,nor_sep_g_ccl1,dra_g_ccl2,ddec_g_ccl2,ang_sep_g_ccl2,pa_g_ccl2,nor_ra_g_ccl2,nor_dec_g_ccl2,nor_sep_g_ccl2,inc_ang_k_g_ccl2,inc_ang_k_ka_ccl2,inc_ang_ka_g_ccl2,max_pa
,mas,mas,,deg,deg,mas,mas,,mas,mas,mas,mas,,mas,deg,,,,mas,mas,,deg,,,,mas,mas,,deg,,,,mas,mas,,deg,,,,mas,mas,,deg,,,,mas,mas,,deg,,,,mas,mas,,deg,,,,mas,mas,,deg,,,,mas,mas,,deg,,,,mas,mas,,mas,mas,mas,mas,,mas,deg,,,,mas,mas,,deg,,,,mas,mas,,deg,,,,mas,mas,,deg,,,,mas,mas,,deg,,,,mas,mas,,deg,,,,mas,mas,,deg,,,,mas,mas,,deg,,,,mas,mas,,deg,,,,mas,mas,mas,mas,mas,mas,mas,,mas,deg,,,,mas,mas,,deg,,,,mas,mas,,deg,,,,mas,mas,,deg,,,,mas,mas,,deg,,,,mas,mas,,deg,,,,mas,mas,,deg,,,,mas,mas,,deg,,,,mas,mas,,deg,,,,deg,deg,deg,
str8,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float

In [8]:
nosou = len(align_sou)

for i in range(nosou):
    mask = (table3_sub["iers_name"] == souname[i])
    table3_sou = table3_sub[mask]

    pai = pa_g[i]
    pa_diff = wrap_angle(table3_sou["PA"] - pai)
    
    mask1 = (np.fabs(pa_diff) < 30) &(table3_sou["F"] != 0)
    table3_sou1 = table3_sou[mask1]
    
    for j in range(len(table3_sou1)):
        print(table3_sou1["iers_name"][j], table3_sou1["F"][j], 
              table3_sou1["ObsEpoch"][j], 
              table3_sou1["r"][j], table3_sou1["PA"][j], "%.2f"%r_g[i], "%.1f"%pai) 

1742-078 1 2014.3586584531142 6.78 182.3 1.20 182.2
1742-078 2 2014.3586584531142 2.75 187.5 1.20 182.2
1742-078 3 2014.3586584531142 1.67 173.1 1.20 182.2
1742-078 4 2014.3586584531142 0.4 199.0 1.20 182.2
1742-078 1 2014.8761122518822 5.1 182.5 1.20 182.2
1742-078 2 2014.8761122518822 3.0 195.7 1.20 182.2
1742-078 4 2014.8761122518822 0.41 197.6 1.20 182.2
1742-078 5 2014.8761122518822 2.06 183.2 1.20 182.2
1742-078 6 2014.8761122518822 1.68 175.1 1.20 182.2
1742-078 1 2015.453798767967 5.03 183.5 1.20 182.2
1742-078 2 2015.453798767967 3.29 193.4 1.20 182.2
1742-078 3 2015.453798767967 1.74 178.0 1.20 182.2
1742-078 4 2015.453798767967 0.46 198.8 1.20 182.2
1742-078 1 2015.678302532512 5.04 183.1 1.20 182.2
1742-078 2 2015.678302532512 3.38 193.6 1.20 182.2
1742-078 3 2015.678302532512 1.78 177.7 1.20 182.2
1742-078 4 2015.678302532512 0.42 197.4 1.20 182.2
1742-078 1 2015.8973305954826 4.97 184.1 1.20 182.2
1742-078 2 2015.8973305954826 3.47 193.4 1.20 182.2
1742-078 3 2015.8973305

Meta information of the table 4.
Columns are as follows: 
- (1) B1950 name, 
- (2) feature number, 
- (3) number of fitted epochs, 
- (4) mean flux density at 15 GHz in milliJansky, 
- (5) mean distance from core feature in milliarcseconds, 
- (6) mean projected distance from core feature in parsec, 
- (7) mean position angle with respect to the core feature in degrees, 
- (8) position angle of velocity vector in degrees, 
- (9) offset between mean position angle and velocity vector position angle in degrees, 
- (10) proper motion in μas yr−1, 
- (11) apparent speed in units of the speed of light, 
- (12) estimated epoch of origin,
- (13) date of reference (middle) epoch used for fit,
- (14) fitted R.A. position with respect to the core at the middle epoch in μas,
- (15) fitted decl. position with respect to the core at the middle epoch in μas. 

A question mark indicates a feature whose motion is not consistent with outward, radial motion but for which the possibility of inward motion and its degree of non-radialness are uncertain.

In [9]:
table4 = Table.read("../data/mojave17/apjab08eet4_mrt.txt",
                    format="ascii.fixed_width_no_header",
                    col_starts=[0, 9, 12, 14, 18, 23, 29, 36, 42, 49, 56, 59, 66,
                                73, 76, 84, 91, 93, 102, 111, 120, 127, 135, 144, 150, 160],
                    col_ends=[8, 11, 13, 17, 22, 28, 35, 41, 48, 55, 58, 65, 72, 75,
                              83, 90, 92, 101, 110, 119, 126, 134, 143, 149, 159, 166],
                    names=["iers_name", "F", "f_F", "Num", "S", "R",
                           "dproj", "theta", "phi", "e_phi", "f_phi",
                           "Diff", "e_Diff", "f_Diff", "pm", "e_pm", "f_pm",
                           "beta-app", "e_beta-app", "tej", "e_tej", "tmid",
                           "alpha-m", "e_alpha-m", "delta", "e_delta-m"])

table4g = table4.group_by("iers_name")
soulist4 = table4g.groups.keys
print("Table 4 offers jet directions for {:d} AGNs.".format(len(soulist4)))

Table 4 offers jet directions for 382 AGNs.


In [10]:
len(join(align_sou, table4g.groups.keys))

20

In [11]:
nosou = len(align_sou)

num = 0

for i in range(nosou):
    mask = (table4["iers_name"] == souname[i])
    table4_sou = table4[mask]

    pai = pa_g[i]
    pa_diff = table4_sou["phi"] - pai

    mask1 = (np.fabs(pa_diff) < 30)
    
    table4_sou1 = table4_sou[mask1]
    pa_diff1 = table4_sou1["phi"] - pai
    
    if len(table4_sou1):
        num += 1
        j = np.argmin(np.fabs(pa_diff1))
        print(table4_sou1["iers_name"][j], table4_sou1["F"][j],
              table4_sou1["phi"][j], "%.2f" % r_g[i], "%.1f" % pai)

print(num)

0003+380 2 120.1 8.42 130.3
0430+052 39 48.0 0.52 75.3
0552+398 2 324.3 0.17 329.0
0743-006 3 225.0 0.98 233.3
0859-140 1 178.4 0.21 204.1
0917+449 4 142.0 0.33 150.1
1157-215 1 324.0 3.97 316.9
1243-072 1 268.1 0.61 256.1
1742-078 2 169.3 1.20 182.2
2134+004 1 96.0 1.41 125.5
10


In [12]:
np.argmin(table4["phi"])

1740

In [13]:
nosou = len(align_sou)

num = 0

for i in range(nosou):
    mask = (table4["iers_name"] == souname[i])
    table4_sou = table4[mask]

    pai = pa_g[i]
    pa_diff = table4_sou["phi"] - pai

    mask1 = (np.fabs(pa_diff) > 150)
    table4_sou1 = table4_sou[mask1]
    
    table4_sou1 = table4_sou[mask1]
    pa_diff1 = table4_sou1["phi"] - pai

    if len(table4_sou1):
        num += 1
        j = np.argmax(np.fabs(pa_diff1))
        print(table4_sou1["iers_name"][j], table4_sou1["F"][j],
              table4_sou1["phi"][j], "%.2f" % r_g[i], "%.1f" % pai)
print(num)

0003+380 6 335.0 8.42 130.3
0133+476 2 342.8 0.15 116.0
0430+052 28 262.9 0.52 75.3
0723-008 5 343.2 1.38 147.7
0743-006 1 42.0 0.98 233.3
0850+581 8 135.9 0.71 336.4
0953+254 4 275.3 0.79 49.2
1038+064 1 137.0 0.38 350.0
1157-215 2 85.0 3.97 316.9
1243-072 4 72.0 0.61 256.1
2251+158 13 342.0 0.49 75.3
11
